#IMPORT THE DATA INTO AN Python object

In [1]:
# Link to download dataset:
# https://rajpurkar.github.io/SQuAD-explorer/

import json

# read file
with open('./data/train-v2.0.json', 'r') as myfile:
    data=myfile.read()

# parse file: Convert .json file into a Python object (its type is unknow, need to check later)
obj = json.loads(data)


# show values

#CONVERT the python object into a dataframe:

In [2]:
# reference: https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/
import pandas as pd

df = pd.DataFrame(obj)

print(df.head())
print(type(df['data']))

  version                                               data
0    v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1    v2.0  {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
2    v2.0  {'title': 'Sino-Tibetan_relations_during_the_M...
3    v2.0  {'title': 'IPod', 'paragraphs': [{'qas': [{'qu...
4    v2.0  {'title': 'The_Legend_of_Zelda:_Twilight_Princ...
<class 'pandas.core.series.Series'>


#STEPS FOR converting the raw data into an easier form of dataset containing columns: 'context', 'question', 'text', 'is_possible', 'answer_start'

## PUT ALL TRIALS INT A STEP TO GET a a_paragraph dataframe from a 'paragraphs' object :

**a function to get a a_paragraph dataframe from a 'paragraphs' object: !! must use variable names different from variable names used in the previous cells in this notebook to make the function as generic as possible**

a_paragraph contains columns ('question',  'is_impossible',  'text', 'answer_start',  'context')

In [3]:
# Need to defines these functions before 'def a_paragraph_return (data)' :
def ExtractFrom_quas_Quest (entry):
  return entry['question']

def ExtractFrom_quas_Id (entry):
  return entry['id']

def ExtractFrom_quas_Answers (entry):
  return entry['answers']

def ExtractFrom_quas_PlausAnswers (entry):
  #print(type(entry))
  #print(entry)
  return entry['plausible_answers']

def ExtractFrom_quas_Is_impossible (entry):
  return entry['is_impossible']

# an entry of 'answers' is a dict => need to extract the infomation out

def ExtractFrom_answers_text (entry):
  return entry[0]['text']
def ExtractFrom_answers_answerStart (entry):
  return entry[0]['answer_start']
# Need to defines these functions before 'def a_paragraph_return (data)' :

def return_a_paragraph (data): # return a dataframe presents a 'paragraphs' object and the number of questions in the 'paragraphs' object

  df_packed_parph= pd.DataFrame(data) # cause the entry contains values of 'qas' as dict => the values in 'qas' column is called to be packed
  #print(len(df_packed_parph)) # number of rows = number of questions in a_paragraph
  df_unpacked_parph=pd.DataFrame(df_packed_parph.loc[:,'qas']) # get all the questions in a paragraph
  '''
  # check all the keys in all dict 'qas's of the dataframe:
  list_of_keys_of_qas=[]
  for i in range (0, len(df_packed_parph)):
     for key in df_packed_parph.loc[i,'qas']:
       list_of_keys_of_qas.append(key)
  # only get all distinct keys in all the 'qas'  of the dataframe:
  distinct_list_of_keys_of_qas = set(list_of_keys_of_qas)
  '''
  # step 1: unpack 'qas' which contains a packed 'answers'
  df_unpacked_parph['question'] = df_unpacked_parph['qas'].apply(ExtractFrom_quas_Quest)
  df_unpacked_parph['id'] = df_unpacked_parph['qas'].apply(ExtractFrom_quas_Id)

  #####
  list_of_answers=[]
  for i in range (0, len(df_packed_parph)):
    if 'plausible_answers' in df_packed_parph.loc[i,'qas'].keys(): # if there is key 'plausible_answers' in this dict 'qas'
                                                                   # if yes, then there is information in 'plausible_answer' instead of key 'answers'   
      answers=ExtractFrom_quas_PlausAnswers(df_packed_parph.loc[i,'qas'])
    else: # if not, means there are information in key 'answers' instead:
      answers=ExtractFrom_quas_Answers(df_packed_parph.loc[i,'qas'])
    list_of_answers.append(answers)
  
  df_unpacked_parph['answers']= list_of_answers
  # there are 'paragraph' objects contain 'plausible_answers' besides 'answers' => need to unpack them
  
  
  df_unpacked_parph['is_impossible'] = df_unpacked_parph['qas'].apply(ExtractFrom_quas_Is_impossible)
  #####
  # must create a new column 'answers'


  # step 2 must be after step 1: unpack 'answers'
  df_unpacked_parph['text'] = df_unpacked_parph['answers'].apply(ExtractFrom_answers_text)
  df_unpacked_parph['answer_start'] = df_unpacked_parph['answers'].apply(ExtractFrom_answers_answerStart)

  # add 'context' to 'questions'
  #df_2.loc[0,'context']
  df_unpacked_parph['context']=df_packed_parph.loc[0,'context']
  return df_unpacked_parph.drop(['qas', 'answers','id'], axis = 1)  # return a dataframe of a 'paragraphs' object

In [5]:
## CHECK THE CODE:
df_1= pd.DataFrame(df.loc[0, 'data'])

# check:
print(df_1.head()) 
print(df_1.groupby('title').size())

# df_1 is a 'data' object:

t= return_a_paragraph(df_1.loc[0, 'paragraphs'])
t

     title                                         paragraphs
0  Beyoncé  {'qas': [{'question': 'When did Beyonce start ...
1  Beyoncé  {'qas': [{'question': 'After her second solo a...
2  Beyoncé  {'qas': [{'question': 'In her music, what are ...
3  Beyoncé  {'qas': [{'question': 'Beyonce's younger sibli...
4  Beyoncé  {'qas': [{'question': 'What town did Beyonce g...
title
Beyoncé    66
dtype: int64


,question,is_impossible,text,answer_start,context
0,When did Beyonce start becoming popular?,False,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,What areas did Beyonce compete in when she was...,False,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,When did Beyonce leave Destiny's Child and bec...,False,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,In what city and state did Beyonce grow up?,False,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,In which decade did Beyonce become famous?,False,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
5,In what R&B group was she the lead singer?,False,Destiny's Child,320,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
6,What album made her a worldwide known artist?,False,Dangerously in Love,505,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
7,Who managed the Destiny's Child group?,False,Mathew Knowles,360,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
8,When did Beyoncé rise to fame?,False,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
9,What role did Beyoncé have in Destiny's Child?,False,lead singer,290,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...


##Get all  'a_paragraph' dataframes,from a 'data' object, into list with function 'a_data'

the first 'data' object has 66 paragraphs, howerver, other 'data' objects have different number of paragraphs => BE CAREFULE for creating the function, MUST MAKE IT AS GENERAL AS POSSIBLE.


In [7]:
print(len(df_1)) # 66 indicates 66 pargraph, df_1 will be the input data for the function 'a_data':



def a_list_of_paragraphs (data): # a list of dataframes 'a_paragraph' presents a 'data' object and total number of questions in the 'data' object
  #total_ques=0
  list_of_paragraphs = []
  for i in range (0, len(data)):
    t= return_a_paragraph(data.loc[i, 'paragraphs'])
    #total_ques= total_ques+ number_of_ques
    list_of_paragraphs.append(t) 
  return list_of_paragraphs

# Check the entries in the 'list_of_paragraphs':
result= a_list_of_paragraphs(df_1)
result[0], type(result[0])

66


(                                             question  ...                                            context
 0            When did Beyonce start becoming popular?  ...  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
 1   What areas did Beyonce compete in when she was...  ...  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
 2   When did Beyonce leave Destiny's Child and bec...  ...  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
 3       In what city and state did Beyonce  grow up?   ...  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
 4          In which decade did Beyonce become famous?  ...  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
 5          In what R&B group was she the lead singer?  ...  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
 6       What album made her a worldwide known artist?  ...  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
 7              Who managed the Destiny's Child group?  ...  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
 

##Get a dataframe presents a 'data' object

In [8]:
# append all the entries in the list_of_paragraphs into a dataframe representing a 'data' object

def a_data (list_paraphs):
  
  data=list_paraphs[0]

  for i in range (1, len(list_paraphs)):
    #print(i)
    data=data.append(list_paraphs[i])
  return data

data = a_data(result)
data

,question,is_impossible,text,answer_start,context
0,When did Beyonce start becoming popular?,False,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,What areas did Beyonce compete in when she was...,False,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,When did Beyonce leave Destiny's Child and bec...,False,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,In what city and state did Beyonce grow up?,False,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,In which decade did Beyonce become famous?,False,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
...,...,...,...,...,...
7,What crowdfunding platform was used in the con...,False,Catapult,1409,"In December, Beyoncé along with a variety of o..."
8,What is the name of the campaign that Beyoncé ...,False,Demand A Plan,108,"In December, Beyoncé along with a variety of o..."
9,What school shooting prompted the creation of ...,False,Sandy Hook Elementary School,280,"In December, Beyoncé along with a variety of o..."
10,What song did Beyoncé donate to the 2012 World...,False,I Was Here,412,"In December, Beyoncé along with a variety of o..."


##Get a dataframe contains all 442 'data' objects in the raw dataset file:

**df_1 is a 'data' object => check how to get it?? at the beginning of the file:**

In [9]:
# check:
print(len(df)) # 442, df contains all the 442 'data' objects

# first 'data' has 66 paragraphs, each has different number of  questions
df_1= pd.DataFrame(df.loc[0, 'data'])
print(df_1), print(df_1.shape)

# second 'data' has 43 paragraphs, each has different number of  questions
df_5= pd.DataFrame(df.loc[5, 'data'])
print(df_5), print(df_5.shape)

442
      title                                         paragraphs
0   Beyoncé  {'qas': [{'question': 'When did Beyonce start ...
1   Beyoncé  {'qas': [{'question': 'After her second solo a...
2   Beyoncé  {'qas': [{'question': 'In her music, what are ...
3   Beyoncé  {'qas': [{'question': 'Beyonce's younger sibli...
4   Beyoncé  {'qas': [{'question': 'What town did Beyonce g...
..      ...                                                ...
61  Beyoncé  {'qas': [{'question': 'What type of accessory ...
62  Beyoncé  {'qas': [{'question': 'Beyonce, during October...
63  Beyoncé  {'qas': [{'question': 'What national disaster ...
64  Beyoncé  {'qas': [{'question': 'Who did Beyonce partici...
65  Beyoncé  {'qas': [{'question': 'Which national event ca...

[66 rows x 2 columns]
(66, 2)
                  title                                         paragraphs
0   Spectre_(2015_film)  {'qas': [{'question': 'Which company made Spec...
1   Spectre_(2015_film)  {'qas': [{'question': 'What group 

(None, None)

**get a list of all 442 dataframes representing 'data' objects in the .json file**

###STEP 1:

In [10]:
# step 1:

# just list of all 'data' objects:
list_of_datas=[]
for i in range (0, len(df)):
  list_of_datas.append(pd.DataFrame(df.loc[i, 'data']))

# check:
print(len(list_of_datas)) # must be 442
list_of_datas[4]

442


,title,paragraphs
0,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'What category of game i...
1,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'Who is the main charact...
2,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'What accolade did Twili...
3,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'What are the three main...
4,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'Many different types of...
5,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'Where is the speaker lo...
6,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'How many dungeon instan...
7,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'What form does Link tak...
8,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'What does AI stand for?...
9,The_Legend_of_Zelda:_Twilight_Princess,{'qas': [{'question': 'Through what can Link's...


###STEP 2:

In [11]:
# Step 2: # list of dataframe of each 'data' object:

list_of_df_datas=[]

for i in range (0, len(list_of_datas)):
  # convert each 'data' object into appropriate dataframe:

  # step 1: get list of paraphs in each 'data':
  input=list_of_datas[i]
  print(i) # CHECK THE ith 'data' ? whether it is processed succesfully??? 
  paragphs = a_list_of_paragraphs(input)

  # step 2: combine the list of paragphs together to have the final datafram of each 'data':
  df_data = a_data(paragphs)

  # step 3: put each dataframe of each 'data' into a list:
  list_of_df_datas.append(df_data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
#CHECK:
list_of_df_datas[1]

,question,is_impossible,text,answer_start,context
0,What was Frédéric's nationalities?,False,Polish and French,182,Frédéric François Chopin (/ˈʃoʊpæn/; French pr...
1,In what era was Frédéric active in?,False,Romantic era,276,Frédéric François Chopin (/ˈʃoʊpæn/; French pr...
2,For what instrument did Frédéric write primari...,False,solo piano,318,Frédéric François Chopin (/ˈʃoʊpæn/; French pr...
3,In what area was Frédéric born in?,False,Duchy of Warsaw,559,Frédéric François Chopin (/ˈʃoʊpæn/; French pr...
4,At what age did Frédéric depart from Poland?,False,20,777,Frédéric François Chopin (/ˈʃoʊpæn/; French pr...
...,...,...,...,...,...
7,What other musician shows to have elements of ...,False,Liszt,269,Chopin's qualities as a pianist and composer w...
0,Who dedicated his 1915 piano Études to Chopin?,False,Debussy,392,"Two of Chopin's long-standing pupils, Karol Mi..."
1,For what publisher to Debussy edit Chopin's mu...,False,Jacques Durand,612,"Two of Chopin's long-standing pupils, Karol Mi..."
2,Who was a student of Chopin's former students ...,False,Raoul Koczalski,197,"Two of Chopin's long-standing pupils, Karol Mi..."


##GET FINAL DATAFRAME FOR THE JSON FILE:

In [16]:
final_df = list_of_df_datas[0]
for i in range (0,len(list_of_df_datas)):
  final_df= final_df.append(list_of_df_datas[i])

In [17]:
# check:
print(final_df.shape) 
print(final_df.loc[0,'question']) # Many rows have the same indices => THIS IS BAD

(131072, 5)
0             When did Beyonce start becoming popular?
0    After her second solo album, what other entert...
0    In her music, what are some recurring elements...
0    Beyonce's younger sibling also sang with her i...
0               What town did Beyonce go to school in?
                           ...                        
0                             When was Descartes born?
0             When did de Sabbata and Gasperini write?
0     What field of physics began in the 19th century?
0               How many quarks and leptons are there?
0    Physics has broadly agreed on the definition o...
Name: question, Length: 19101, dtype: object


In [18]:
# reset the indices of final_df 
final_df.reset_index(drop=True, inplace=True)

In [19]:
#CHECK:
final_df

,question,is_impossible,text,answer_start,context
0,When did Beyonce start becoming popular?,False,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,What areas did Beyonce compete in when she was...,False,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,When did Beyonce leave Destiny's Child and bec...,False,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,In what city and state did Beyonce grow up?,False,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,In which decade did Beyonce become famous?,False,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
...,...,...,...,...,...
131067,Physics has broadly agreed on the definition o...,True,matter,485,"The term ""matter"" is used throughout physics i..."
131068,Who coined the term partonic matter?,True,Alfvén,327,"The term ""matter"" is used throughout physics i..."
131069,What is another name for anti-matter?,True,Gk. common matter,350,"The term ""matter"" is used throughout physics i..."
131070,Matter usually does not need to be used in con...,True,a specifying modifier,529,"The term ""matter"" is used throughout physics i..."


##SAVE THE FINAL DATASET:

In [20]:
# sample down the dataset: only takes 10000 samples from the dataset
# should randomize the dataset before sampling down the dataset # not done yet
final=final_df.loc[0:10000,:]

In [21]:
final

,question,is_impossible,text,answer_start,context
0,When did Beyonce start becoming popular?,False,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,What areas did Beyonce compete in when she was...,False,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,When did Beyonce leave Destiny's Child and bec...,False,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,In what city and state did Beyonce grow up?,False,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,In which decade did Beyonce become famous?,False,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
...,...,...,...,...,...
9996,What famous World War II battle was the Canadi...,False,the Normandy Landings,166,Battles which are particularly notable to the ...
9997,What effort was the Canadian Military known fo...,False,the strategic bombing of German cities,288,Battles which are particularly notable to the ...
9998,What Battle in France was the Canadian Militar...,False,the Battle of Vimy Ridge,72,Battles which are particularly notable to the ...
9999,What country was the latest Canadian Military ...,False,Croatia,377,Battles which are particularly notable to the ...


In [22]:
final.to_csv("data/final.tsv", sep="\t", index=False)